In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
%matplotlib inline

/home/fsguzi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# read

In [2]:
meta = [('a','20180606'),('b','20180726')]
w = meta[1]

path = '/media/fsguzi/shits/tianchi/schedule/'
interfere = pd.read_csv(path+'scheduling_preliminary_%s_app_interference_%s.csv'%w,header=None)
app = pd.read_csv(path+'scheduling_preliminary_%s_app_resources_%s.csv'%w,header=None)
instance = pd.read_csv(path+'scheduling_preliminary_%s_instance_deploy_%s.csv'%w,header=None)
machine = pd.read_csv(path+'scheduling_preliminary_%s_machine_resources_%s.csv'%w,header=None)

interfere.columns = ['App1','App2','k']
app.columns = ['App','cpu','mem','disk','P','M','PM']
instance.columns = ['Ins','App','Mac']
machine.columns = ['Mac','cpu','mem','disk','P','M','PM']

pos = {'Ins':5,'App':4,'Mac':8}
for df in [interfere,app,instance,machine]:
    for cname in df.columns:
        if cname in pos or cname[:3]=='App':
            df.loc[:,cname] = df.loc[:,cname].str[pos[cname[:3]]:].fillna(0).astype('int')

app['App']-=1
instance['App']-=1
interfere[['App1','App2']] -=1
instance['Mac']-=1

cpu = app['cpu'].str.split('|',expand=True).astype(np.float64)
mem = app['mem'].str.split('|',expand=True).astype(np.float64)
app = pd.concat([app.iloc[:,[0,3,4,5,6]],cpu,mem],axis=1)


#instance = instance.sort_values('disk',ascending=False)
info = instance.iloc[:,1].values

machine=machine.astype(float)
#machine = machine.sort_values('disk',ascending=False)
machine = np.concatenate([machine.iloc[:,3:].values,np.tile(machine.cpu.values,(98,1)).T,np.tile(machine.mem.values,(98,1)).T],axis=1)

machine = machine[::-1]

In [3]:
app_count = pd.DataFrame(instance.groupby('App').Ins.count()).reset_index()
interfere['n'] = interfere.App2.apply(lambda x:app_count.iloc[x,1])
interfere.loc[(interfere.App1==interfere.App2),'k'] +=1 
interfere = interfere[interfere.n>interfere.k]
interfere = interfere.iloc[:,:3].sort_values(['App1','App2']).reset_index(drop=True)

app_inter_map_1 = np.zeros((9338,len(interfere)),dtype=int)
app_inter_map_2 = np.zeros((9338,len(interfere)),dtype=int)
mac_inter_limit = np.zeros(len(interfere),dtype=int)

for idx in range(len(interfere)):
    a1,a2,k = interfere.iloc[idx,:3]    
    app_inter_map_1[a1,idx] = True
    app_inter_map_2[a2,idx] = 1
    mac_inter_limit[idx] = k
    
def get_int_num(x):
    out = interfere[(interfere.App1==x.App)|(interfere.App2==x.App)]
    return out.index.tolist() if len(out)>0 else []
in_c = app.apply(get_int_num,axis=1).tolist()

app = app.iloc[:,1:].values

# schedule

In [4]:
info2 = []
for a in np.argsort(app[:,0])[::-1]:
    for _ in range(app_count.Ins[a]):
        info2.append(a)

In [4]:
tf.reset_default_graph()
sess = tf.Session()

bk = np.zeros_like(machine,dtype='float64')
bk_1 = np.zeros((6000,len(interfere))).T
bk_2 = np.zeros((6000,len(interfere))).T

plan_a = [[] for _ in range(6000)]

flag_half = 0
flag = 0
remaining =[]

machine_cpu = machine[:,4:102]
score = np.zeros((6000,98))


#variables
score_tf = tf.Variable(score,dtype='float32')
bk_tf = tf.Variable(bk,dtype='float32')


#constant
machine_tf = tf.constant(machine,dtype='float32')
machine_cpu_tf = tf.constant(machine_cpu,dtype='float32')


#placeholders
row_tf = tf.placeholder(shape=(200,),dtype='float32')


would_be_tf = row_tf + bk_tf
yn_r_tf = tf.reduce_all(would_be_tf <= machine_tf,axis=1)


    
bk_1_tf = tf.Variable(bk_1,dtype='uint8')
bk_2_tf = tf.Variable(bk_2,dtype='uint8')
mac_inter_limit_tf = tf.constant(mac_inter_limit,dtype='uint8')

in_cols_tf = tf.placeholder(dtype='int32')
inter1_tf = tf.placeholder(dtype='uint8')
inter2_tf = tf.placeholder(dtype='uint8')
mac_inter_limit_tf_slice = tf.placeholder(dtype='uint8')

bk_1_tf_slice = tf.gather(bk_1_tf,in_cols_tf,axis=0)
bk_2_tf_slice = tf.gather(bk_2_tf,in_cols_tf,axis=0)
#mac_inter_limit_tf_slice = tf.gather(mac_inter_limit_tf,in_cols_tf)


yn_i_tf_1 = tf.bitwise.bitwise_or(inter1_tf,bk_1_tf_slice)

yn_i_tf_2 = inter2_tf + bk_2_tf_slice

yn_i_tf = (yn_i_tf_1 * yn_i_tf_2) <= mac_inter_limit_tf_slice
yn_i_tf = tf.reduce_all(yn_i_tf,axis=0)

put_tf = tf.placeholder(dtype='int32')
scatter_idx_tf = tf.placeholder(dtype='int32')
new_bk_1_tf = yn_i_tf_1[:,put_tf]

update_bk_tf = tf.scatter_add(bk_tf,put_tf,row_tf)
update_bk_1_tf = tf.scatter_nd_update(bk_1_tf,scatter_idx_tf,new_bk_1_tf)
update_bk_2_tf = tf.scatter_nd_add(bk_2_tf,scatter_idx_tf,inter2_tf[:,0])

update_bk_tf_op = sess.graph.get_operation_by_name('ScatterAdd')
update_bk_1_tf_op = sess.graph.get_operation_by_name('ScatterNdUpdate')
update_bk_2_tf_op = sess.graph.get_operation_by_name('ScatterNdAdd')

In [14]:
sess.close()
del sess

In [ ]:
sess.run(tf.global_variables_initializer())
start = time.clock()
a= time.clock()
for i,app_num in enumerate(info):
    row = app[app_num]
    in_cols = in_c[app_num]
    #in_cols = [_*80 for _ in range(20)]
    l=len(in_cols)
    inter1 = app_inter_map_1[app_num,in_cols].reshape(l,1)
    inter2 = app_inter_map_2[app_num,in_cols].reshape(l,1)
    
    mac_inter_limit_slice = mac_inter_limit[in_cols].reshape(l,1)
    
    yn_r, yn_i = sess.run([yn_r_tf,yn_i_tf],feed_dict={row_tf:row,
                                          in_cols_tf:in_cols,
                                     inter1_tf:inter1,
                                     inter2_tf:inter2,
                                    mac_inter_limit_tf_slice:mac_inter_limit_slice })
        
   
    #locations of feasible machines
    put = np.argwhere(yn_r&yn_i)[:,0]
    if len(put)==0:
        flag+=1
    else:
        put = put[0]
        
        sess.run([update_bk_tf_op],feed_dict={row_tf:row,
                                                 put_tf:put })
        if l > 0:
            
            scatter_idx = [[col,put] for col in in_cols]
            sess.run([update_bk_1_tf_op,update_bk_2_tf_op],
                                     feed_dict={row_tf:row,
                                                 put_tf:put,
                                                in_cols_tf:in_cols,
                                                 inter1_tf:inter1,
                                                 inter2_tf:inter2,
                                               scatter_idx_tf:scatter_idx})
            
    if i%5000==0:
        b = time.clock()
        print(i,b-a)
        a = time.clock()
        
end = time.clock()
print(flag)
print(end-start)
final = sess.run(bk_tf)
final_u = final/machine
final_s = np.piecewise(final_u,[final_u>0,final_u>0.5],[1,lambda x:1 + 10*(np.exp(x-0.5)-1)])
print(final_s.sum()/98)

0 0.041156999999998334
5000 34.179863999999995


#  relocate

In [181]:
# list(6000) of list of app # in that indexed-machine       
#plan_a 

#array(6000,9338) of how many of app# on each machine(plan)
plan_a_c = np.zeros((6000,9338),dtype=int)
for m_ in range(6000):
    for a_ in plan_a[m_]:
        plan_a_c[m_,a_] += 1
        
#array(6000,9338) of how many of app# on each machine(already)
cur_a = np.zeros((6000,9338),dtype=int)
for a_,m_ in instance.loc[:,['App','Mac']].values:
    if m_>=0:
        cur_a[m_,a_] +=1
        
#dict(-1 to 5999,machines) of dict(0-9337,app#) of list showing what ins# each machine already carring
inventory ={m_:{} for m_ in range(-1,6000)}
for i_,(a_,m_) in enumerate(instance.loc[:,['App','Mac']].values):
    if inventory[m_].get(a_,[]) == []:
        inventory[m_][a_] = [i_]
    else:
        inventory[m_][a_].append(i_)
        
#order in which planned machine alocations are being fulfilled
order = plan_a_c.sum(axis=1).argsort()[::-1].tolist()

In [182]:
o = 0

kill = 0
innocent = 0
iteration=0

shuchu = []

while len(order)>0:
    iteration+=1
    success=0
    total = len(order)
    neworder = list(order)
    
    #put in empty machines
    for i in order:
        plan_one = plan_a[i]
        if len(plan_one) ==0:
            success+=1
            neworder.remove(i)
            continue
        half = (i//3000)*3000
        look = cur_a[half:(half+3000)].sum(axis=1)
        put = np.argwhere(look==0)
        if len(put)==0:

            continue
        else:
            put = put[0,0]+half
        for a_ in plan_one:
            o+=1
            #put in 
            cur_a[put,a_] -= 1


            #take out
            take = np.argwhere(cur_a[:,a_]>0)
            if len(take) ==0:
                innocent+=1
                
                take_ins = inventory[-1][a_][0]
                del inventory[-1][a_][0]
                shuchu.append([take_ins,put])
                
                continue
            else:
                best = np.argmin(cur_a[take[:,0]].sum(axis=1))
                take = take[best,0]

                cur_a[take,a_] -=1
                
                take_ins = inventory[take][a_][0]
                del inventory[take][a_][0]
                shuchu.append([take_ins,put])
                
                if cur_a[take].sum()==0:
                    kill+=1  
        neworder.remove(i)
        success+=1
    
    
    #match machines with only 1 instance
    ones = np.argwhere((cur_a.sum(axis=1)==1))[:,0].tolist()
    for i in ones:
        order_ = np.array(neworder)
        half = (i//3000)*3000

        #find which app
        cur_row = cur_a[i]
        if cur_row.sum()==0:
            
            continue
        only_a = np.argwhere(cur_row>0)[0,0]

        #look order
        look_order = order_[(order_<half+3000) & (order_>=half)]
        if len(look_order) ==0:
            continue
        qualified_order_sub = np.argwhere(plan_a_c[look_order,only_a]>=1)[:,0]
        if len(qualified_order_sub) == 0:
            continue
        look_order = look_order[qualified_order_sub]
        put = np.argmax(plan_a_c[look_order].sum(axis=1))

        fill = look_order[put]
        
        
        plan_one = plan_a[fill]

        for a_ in plan_one:
            o+=1
            #put in 
            if a_ == only_a and cur_a[i,a_] > -1:
                cur_a[i,a_] = -1
                
                take_ins = inventory[i][a_][0]
                del inventory[i][a_][0]
                shuchu.append([take_ins,i])
                
                continue

            else:
                cur_a[i,a_] -= 1


            #take out
            take = np.argwhere(cur_a[:,a_]>0)
            if len(take) ==0:
                innocent+=1
                
                take_ins = inventory[-1][a_][0]
                del inventory[-1][a_][0]
                shuchu.append([take_ins,i])
                
                continue
            else:
                best = np.argmin(cur_a[take[:,0]].sum(axis=1))
                take = take[best,0]
                
                cur_a[take,a_] -=1
                
                take_ins = inventory[take][a_][0]
                del inventory[take][a_][0]
                shuchu.append([take_ins,i])
                
                if cur_a[take].sum()==0:
                    kill+=1  
    
        neworder.remove(fill)
        success+=1
    order = list(neworder)
    print(iteration,total,success)

1 6000 5296
2 704 704


In [184]:
shuchu = np.array(shuchu)
out = pd.DataFrame('inst_'+instance.Ins[shuchu[:,0]].astype(str)).reset_index(drop=True)
out['m'] = 'machine_'+pd.Series((shuchu[:,1]+1).astype(str))
out.to_csv(r'E:\data\kaggle\machine\b.csv',header=False,index=False,encoding='utf-8')

# out

In [211]:
out = pd.DataFrame('inst_'+instance.Ins.astype(str))
out['m'] = 'machine_'+(result.mac+1).astype(str)
out.to_csv(r'E:\data\kaggle\machine\asd.csv',header=False,index=False)

In [114]:
a = open(path+r'\a.csv')
b = open(path+r'\dsa.csv')
w = open(path+r'\submit_1.csv','w')

a_content = a.read()
b_content = b.read()
w.write(a_content+'#\n'+b_content)

a.close()
b.close()
w.close()

In [145]:
(9619+6206)/2

7912.5